# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

In [0]:
# TODO - we'll be doing this live, taking requests
# and reproducing what it is to look up and learn things

## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

In [0]:
# TODO - your code here
# Use what we did live in lecture as an example


Some research about gun data


https://colab.research.google.com/drive/1EbSZuW-SM5EBVly-ZQyGXxFbHnXPmJNE

### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.

I searched for and found a data set and notebook that studied something I find interesting -- Gun Data. I downloaded this information and opened it into a Google Colab so that I could explore what had been done in more detail. I imported that data that had been used in the original study and followed each step to make sure I got the same results.  

2.  What was the most challenging part of what you did?

I came across a number of errors when running the cells, and had to look into what was causing them and how to fix the errors. Most of them were caused by not having libraries imported, or not being able to access the CSVs that were used in the original study.

3.  What was the most interesting thing you learned?

That once the data is cleaned and organized, it will be much easier to try multiple experiments and see what happens to the results.

4.  What area would you like to explore with more time?

This particular data set used bar charts to depict the data. While this is a clear way to present data, I would like to explore different methods of data visualization.




## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about.